In [21]:
import cv2, torch, torchvision

In [22]:
import model
import torch.nn as nn

In [23]:
module = model.Resnet(3,model.ResBottleneckBlock,[3,4,6,3], useBottleneck=True).to('cuda')

In [24]:
resnet50

Resnet(
  (layer0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (layer1): Sequential(
    (conv2_1): ResBottleneckBlock(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
        (1): AvgPool2d(kernel_size=3, stride=1, padding=0)
        (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [25]:
device= torch.device('cuda')

In [26]:
input = cv2.imread('test.jpg')
input = cv2.flip(input,1)
input = cv2.resize (input, (128,128))
input = torch.tensor(input.transpose([2, 0, 1]), dtype=torch.float,device = device)


In [29]:
output=resnet50(input.unsqueeze(0))

In [43]:
output=resnet50.prediction(output)

In [32]:
input.unsqueeze(0).shape

torch.Size([1, 3, 128, 128])

In [44]:
output.shape

torch.Size([1, 21, 32, 32])

In [16]:
from torch.optim import Adam
 
# Define the loss function with Classification Cross-Entropy loss and an optimizer with Adam optimizer
loss_fn = nn.MSELoss()
optimizer = Adam(module.parameters(), lr=0.001, weight_decay=0.0001)

In [20]:
input = torch.randn(1,3,128,128, requires_grad=True)
target = torch.randn(1,21,32,32)
bla=module(input)
output = loss_fn(bla, target)
output.backward()

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor